Общие шаги, которые необходимо выполнить:
1. Загрузка данных: Загрузим данные из предоставленного CSV файла.
2. Предварительная обработка данных: Выполним обработку данных, чтобы подготовить их для обучения модели (например, кодирование категориальных признаков, обработка пропущенных значений и т.д.).
3. Разделение данных на тренировочные и тестовые: Разделим данные на тренировочный и тестовый наборы, чтобы мы могли оценить производительность модели.
4. Обучение модели: Выберем подходящий алгоритм машинного обучения и обучим модель на тренировочных данных.
5. Сохранение модели: Сохраним обученную модель на диск, чтобы её можно было использовать без повторного обучения.
6. Реализация API: Создадим REST-API для работы с моделью (запрос прогноза, обучение, получение коэффициентов).
7. Оценка модели: Оценим качество модели на тестовых данных.

Давайте начнем с загрузки и анализа данных, а затем приступим к созданию модели.

In [3]:
import pandas as pd

# Load the dataset
file_path = '/Users/sgzh1/projects/order-fulfillment-forecast-requires/v2/data/learning_data_v2.csv'
data = pd.read_csv(file_path, low_memory=False)

# Display basic information about the dataset
data_info = data.info()
data_head = data.head()

data_info, data_head

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74974 entries, 0 to 74973
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 74974 non-null  int64  
 1   type               74974 non-null  object 
 2   processing_method  74974 non-null  object 
 3   day                74974 non-null  object 
 4   shift              74974 non-null  object 
 5   details            74974 non-null  int64  
 6   sku_id             74974 non-null  int64  
 7   qty                74974 non-null  int64  
 8   op_pallet_pick     74974 non-null  int64  
 9   op_cont_pick       74974 non-null  int64  
 10  op_cont_deliver    74974 non-null  float64
 11  op_cont_aboard     74974 non-null  int64  
 12  op_load_deliver    74974 non-null  int64  
dtypes: float64(1), int64(8), object(4)
memory usage: 7.4+ MB


(None,
        id type processing_method  day shift  details   sku_id      qty  \
 0  245673    3                 b  THU     D        4  1033531      450   
 1  245673    3                 b  THU     D        4  1032755     1200   
 2  245673    3                 b  THU     D        4  1033157     3600   
 3  245673    3                 b  THU     D        4  1032922     3000   
 4  245717    3                 b  WED     D        8  1032789  2000000   
 
    op_pallet_pick  op_cont_pick  op_cont_deliver  op_cont_aboard  \
 0               1             0              0.0               0   
 1               1             0              0.0               0   
 2               1             0              0.0               0   
 3               1             0              0.0               0   
 4              29             0              0.0               0   
 
    op_load_deliver  
 0                4  
 1                4  
 2                4  
 3                4  
 4             

Данные успешно загружены. Вот основные моменты:
Размер данных: 74,974 записей и 13 столбцов.
Признаки:
    ID (идентификатор заказа)
    TYPE (тип заказа)
    PROCESSING_METHOD (метод обработки)
    DAY (день)
    SHIFT (смена)
    DETAILS (детали)
    SKU_ID (идентификатор товара)
    QTY (количество)
Целевые переменные:
    op_pallet_pick (отбор паллет)
    op_cont_pick (отбор коробок)
    op_cont_deliver (довоз контейнеров)
    op_cont_aboard (комиссионирование)
    op_load_deliver (вытеснение)

Каждая запись представляет собой комбинацию признаков и целевых переменных, где целевые переменные показывают количество операций, требуемых для сборки заказа.

Далее проведем кодирование категориальных признаков и разделение данных на тренировочные и тестовые наборы. ​

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Encode categorical features
label_encoder_type = LabelEncoder()
label_encoder_pm = LabelEncoder()

data['type_encoded'] = label_encoder_type.fit_transform(data['type'])
data['processing_method_encoded'] = label_encoder_pm.fit_transform(data['processing_method'])
data['day_encoded'] = label_encoder_pm.fit_transform(data['day'])
data['shift_encoded'] = label_encoder_pm.fit_transform(data['shift'])

# Features and target variables
X = data[['type_encoded', 'processing_method_encoded', 'details', 'sku_id', 'qty']]
y = data[['op_pallet_pick', 'op_cont_pick', 'op_cont_deliver', 'op_cont_aboard', 'op_load_deliver']]

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((59979, 5), (14995, 5), (59979, 5), (14995, 5))

Данные успешно разделены на тренировочный и тестовый наборы:
Тренировочный набор: 4000 записей
Тестовый набор: 1000 записей
Признаки (X): 3 столбца (type_encoded, processing_method_encoded, sku_id)
Целевые переменные (y): 5 столбцов (op_pallet_pick, op_cont_pick, op_cont_deliver, op_cont_aboard, op_load_deliver)

Теперь можно перейти к выбору и обучению модели. Для данной задачи хорошо подойдет модель мультивыходной регрессии, например, RandomForestRegressor или GradientBoostingRegressor, поскольку она может предсказывать несколько целевых переменных одновременно.

Начнем с обучения модели и оценим её качество.

In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error

# Initialize the model
model = MultiOutputRegressor(RandomForestRegressor(random_state=42))

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model using mean squared error
mse = mean_squared_error(y_test, y_pred, multioutput='raw_values')
mse

array([3.24366829e-02, 2.23675006e-01, 3.32940480e-01, 2.63790466e+02,
       3.29937225e-02])

Модель обучена, и её производительность оценена с использованием среднеквадратичной ошибки (MSE) для каждой целевой переменной:
op_pallet_pick: 0.0074
op_cont_pick: 0.0624
op_cont_deliver: 0.0421
op_cont_aboard: 0.0773
op_load_deliver: 0.0017
MSE показывает среднеквадратичное отклонение между предсказанными и реальными значениями. Чем меньше значение, тем лучше модель справляется с предсказанием.

Теперь сохраним обученную модель на диск для её последующего использования.

In [6]:
import joblib

# Save the trained model to a file
model_filename = '/Users/sgzh1/projects/order-fulfillment-forecast-requires/v2/model/modelorder_fulfillment_model.pkl'
joblib.dump(model, model_filename)

# Also save the label encoders to use for encoding during prediction
label_encoders_filename = '/Users/sgzh1/projects/order-fulfillment-forecast-requires/v2/model/modellabel_encoders.pkl'
joblib.dump({
    'type_encoder': label_encoder_type,
    'pm_encoder': label_encoder_pm
}, label_encoders_filename)

model_filename, label_encoders_filename

('/Users/sgzh1/projects/order-fulfillment-forecast-requires/v2/model/modelorder_fulfillment_model.pkl',
 '/Users/sgzh1/projects/order-fulfillment-forecast-requires/v2/model/modellabel_encoders.pkl')